# 🧹📓 **02 - Generación de datos aumentados**


Este notebook crea versiones aumentadas del dataset **sin depender de librerías avanzadas**. Se utilizan `cv2`, `numpy`, `PIL` y utilidades estándar para producir flips, cambios de color y desenfoques de movimiento.

In [34]:
import os, glob, random, shutil
from pathlib import Path
import numpy as np
import cv2
from tqdm import tqdm

In [35]:
# Rutas de origen (data/raw) y destino (data/interim)
RAW_IMG_DIR = Path('data/raw/train/images')
RAW_LBL_DIR = Path('data/raw/train/labels')

AUG_IMG_DIR = Path('data/interim/train_aug/images')
AUG_LBL_DIR = Path('data/interim/train_aug/labels')
if AUG_IMG_DIR.exists():
    shutil.rmtree(AUG_IMG_DIR)
AUG_IMG_DIR.mkdir(parents=True, exist_ok=True)
AUG_LBL_DIR.mkdir(parents=True, exist_ok=True)
print('Augmentations se guardarán en:', AUG_IMG_DIR.parent)

Augmentations se guardarán en: data\interim\train_aug


In [ ]:
def hflip(img, labels):
    # Horizontal flip
    # Cambia las coordenadas x de las etiquetas 
    flipped = cv2.flip(img, 1)
    h, w, _ = img.shape
    new_labels = []
    # Las coordenadas x se invierten
    # x_new = 1 - x para invertir horizontalmente 
    for cls, x, y, bw, bh in labels:
        x_new = 1 - x
        new_labels.append((cls, x_new, y, bw, bh)) # cls, x, y, bw, bh 
    return flipped, new_labels


def hsv_jitter(img):
    # HSV jitter
    # Cambia el tono, saturación y valor de la imagen
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV).astype(np.float32)
    h, s, v = cv2.split(hsv)
    h = (h + random.uniform(-5,5)) % 180 # 180 grados para el tono 
    s *= random.uniform(0.8, 1.2) # Saturación entre 0.8 y 1.2 
    v *= random.uniform(0.8, 1.2) # Valor entre 0.8 y 1.2 
    hsv_aug = cv2.merge([h, s, v]) # Combina los canales modificados 
    hsv_aug = np.clip(hsv_aug,0,255).astype(np.uint8) # Asegura que los valores estén en el rango correcto 
    return cv2.cvtColor(hsv_aug, cv2.COLOR_HSV2BGR) # Convierte de nuevo a BGR 

def motion_blur(img, k=5):
    # Motion blur
    # Aplica un desenfoque de movimiento a la imagen
    kernel = np.zeros((k, k)) 
    kernel[int((k-1)/2), :] = np.ones(k) # Línea horizontal de unos 
    kernel /= k # Normaliza el kernel 
    return cv2.filter2D(img, -1, kernel)

In [ ]:
def read_labels(txt):
    labels=[]
    with open(txt) as f:
        for line in f: # Leer cada línea del archivo 
            cls,x,y,w,h=map(float, line.split()) # cls, x, y, w, h = map(float, line.split())
            # Convertir las coordenadas a float y agregar a la lista de etiquetas 
            labels.append((int(cls),x,y,w,h))
    return labels

def write_labels(txt, labels):
    with open(txt,'w') as f: 
        for cls,x,y,w,h in labels: # cls, x, y, w, h = labels:
            # Escribir cada etiqueta en el archivo 
            f.write(f"{cls} {x:.6f} {y:.6f} {w:.6f} {h:.6f}\n")

In [ ]:
images = glob.glob(str(RAW_IMG_DIR/'*.jpg'))
for img_path in tqdm(images, desc='Augmentando'):
    base = Path(img_path).stem # Obtener el nombre base del archivo sin extensión
    # Verificar si el archivo de etiquetas existe 
    lbl_path = RAW_LBL_DIR/f'{base}.txt'
    if not lbl_path.exists():
        continue
    # copiar original
    shutil.copy(img_path, AUG_IMG_DIR/f'{base}.jpg') # Copiar imagen original
    # copiar etiquetas originales 
    shutil.copy(lbl_path, AUG_LBL_DIR/f'{base}.txt')

    img = cv2.imread(img_path)
    labels = read_labels(lbl_path)

    # flip
    img_f, lab_f = hflip(img, labels)
    cv2.imwrite(str(AUG_IMG_DIR/f'{base}_flip.jpg'), img_f)
    write_labels(AUG_LBL_DIR/f'{base}_flip.txt', lab_f)

    # hsv
    img_h = hsv_jitter(img)
    cv2.imwrite(str(AUG_IMG_DIR/f'{base}_hsv.jpg'), img_h)
    shutil.copy(lbl_path, AUG_LBL_DIR/f'{base}_hsv.txt')

    # blur
    img_b = motion_blur(img)
    cv2.imwrite(str(AUG_IMG_DIR/f'{base}_blur.jpg'), img_b)
    shutil.copy(lbl_path, AUG_LBL_DIR/f'{base}_blur.txt')
print('Augmentations completas.')

Augmentando: 100%|██████████| 400/400 [02:11<00:00,  3.04it/s]

Augmentations completas.
